In [38]:
pip install tensorflow-gpu


  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 126, in _parse_requirement_details
      

In [39]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  2


In [40]:
# Import necessary libraries
import numpy as np
import pprint
from keras.datasets import mnist
import cupy 
import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [41]:


class ANN_Model:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def train(self, X, Y, weights, epochs=1, validation_data=None, batch_size=200):
        self.set_weights(weights)
        best_val_loss = float('inf')

        for epoch in range(epochs):
            for batch_start in range(0, len(X), batch_size):
                batch_end = min(batch_start + batch_size, len(X))
                X_batch = X[batch_start:batch_end]
                Y_batch = Y[batch_start:batch_end]
                predicted = self.forward(X_batch)
                loss = self.compute_loss(predicted, Y_batch)


            if validation_data is not None:
                val_loss = self.compute_loss(self.forward(validation_data[0]), validation_data[1])
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                else:
#                     print(f"Validation loss not improving. Stopping early at epoch {epoch}")
                    break

    def predict(self, X):
        return self.forward(X)

    def compute_loss(self, predicted, actual):
        epsilon = 1e-15
        loss = -np.mean(actual * np.log(predicted + epsilon))
        return loss

    def calculate_accuracy(self, predicted, actual):
        predicted_classes = np.argmax(predicted, axis=1)
        true_classes = np.argmax(actual, axis=1)
        accuracy = np.mean(predicted_classes == true_classes)
        return accuracy

    def set_weights(self, weights):
        index = 0
        for layer in self.layers:
            if hasattr(layer, 'weights'):
                layer_size = np.prod(layer.weights.shape)
                layer.weights = weights[index:index+layer_size].reshape(layer.weights.shape)
                index += layer_size

    def get_weights_and_biases(self):
        parameters = {}
        for i, layer in enumerate(self.layers):
            if hasattr(layer, 'weights') and hasattr(layer, 'biases'):
                parameters[f'weights{i+1}'] = layer.weights
                parameters[f'biases{i+1}'] = layer.biases
        return parameters

class FlattenLayer:
    def forward(self, X):
        return X.reshape(X.shape[0], -1)

class DenseLayer:
    def __init__(self, input_size, output_size, activation='relu'):
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2.0 / input_size)
        self.biases = np.zeros((1, output_size))
        self.activation = activation

    def forward(self, X):
        self.X = X
        self.Z = np.dot(X, self.weights) + self.biases
        if self.activation == 'relu':
            return np.maximum(0, self.Z)
        else:
            return self.Z

class DropoutLayer:
    def __init__(self, dropout_rate):
        self.dropout_rate = dropout_rate

    def forward(self, X):
        self.mask = np.random.rand(*X.shape) < self.dropout_rate
        return X * self.mask

In [42]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data (Normalization)
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [43]:
def fitness_function(weights):
    model = ANN_Model()
    model.add(FlattenLayer())
    model.add(DenseLayer(input_size=784, output_size=512, activation='relu'))
    model.add(DropoutLayer(dropout_rate=0.2))
    model.add(DenseLayer(input_size=512, output_size=10, activation='softmax'))
    model.set_weights(weights)
    model.train(x_train, y_train, weights, epochs=1, validation_data=(x_test, y_test))
    predictions = model.predict(x_test)
    accuracy = model.calculate_accuracy(predictions, y_test)
    return 1 / (accuracy + 0.00001)


In [44]:
GENERATIONS=4000
vector_len=4019200
batch_size=200
crossover='interm'
mutation='1/rand'
method='JADE'
POP_TOPOLOGY="one population"
population_size=300

In [45]:
import numpy as np

# Xavier Initialization for vectors
def xavier_init(shape):
    """
    Args:
    - shape (int): Size of the weight vector to initialize.
    """
    var = 2.0 / shape  # Variance
    return np.random.randn(shape) * np.sqrt(var)  # Sample from Gaussian distribution

# He Initialization for vectors
def he_init(shape):
    """
    Args:
    - shape (int): Size of the weight vector to initialize.
    """
    var = 2.0 / shape  # Variance
    return np.random.randn(shape) * np.sqrt(var)  # Sample from Gaussian distribution

# LeCun Initialization for vectors
def lecun_init(shape):
    """
    Args:
    - shape (int): Size of the weight vector to initialize.
    """
    limit = np.sqrt(3.0 / shape)  # Limit
    return np.random.uniform(-limit, limit, shape)  # Sample from uniform distribution

# Orthogonal Initialization for vectors
def orthogonal_init(shape):
    """
    Args:
    - shape (int): Size of the weight vector to initialize.
    """
    a = np.random.normal(0.0, 1.0, shape)  # Generate random vector
    q, _ = np.linalg.qr(a)  # Compute QR decomposition
    return q  # Return orthogonal vector


In [46]:
class chromosome:
    Genes = None
    Fitness = None
    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness

In [47]:

# first population using xavier_init initialization 
population=[]
for i in range(population_size):
    print(f"Initializing chromosome {i+1}...")
    genes = xavier_init(vector_len)
    fitness = fitness_function(genes)
    print(f"Chromosome {i+1} Fitness: {fitness}")
    population.append(chromosome(genes, fitness))


Initializing chromosome 1...


/tmp/ipykernel_128/2387734017.py:39: RuntimeWarning: invalid value encountered in log
  loss = -np.mean(actual * np.log(predicted + epsilon))


Chromosome 1 Fitness: 9.433072351664938
Initializing chromosome 2...
Chromosome 2 Fitness: 9.59600806064677
Initializing chromosome 3...
Chromosome 3 Fitness: 10.856584518510477
Initializing chromosome 4...
Chromosome 4 Fitness: 10.308215647871354
Initializing chromosome 5...
Chromosome 5 Fitness: 10.536297545042673
Initializing chromosome 6...
Chromosome 6 Fitness: 10.297600659046442
Initializing chromosome 7...
Chromosome 7 Fitness: 10.151253679829459
Initializing chromosome 8...
Chromosome 8 Fitness: 11.122233344455568
Initializing chromosome 9...
Chromosome 9 Fitness: 8.298066550493736
Initializing chromosome 10...
Chromosome 10 Fitness: 10.844810758052272
Initializing chromosome 11...
Chromosome 11 Fitness: 10.329511414110113
Initializing chromosome 12...
Chromosome 12 Fitness: 10.340192327577293
Initializing chromosome 13...
Chromosome 13 Fitness: 9.939369843951894
Initializing chromosome 14...
Chromosome 14 Fitness: 10.009008107296568
Initializing chromosome 15...
Chromosome 15 

In [48]:
import numpy as np
import matplotlib.pyplot as plt

def jade(fitness_function, dim=4019200, NP=300, CR=0.9, F=0.8, max_gen=4000):
    memory_CR, memory_F = [], []
    best_fitness_progress = []  # List to store the best fitness value at each generation
    avg_fitness_progress = []   # List to store the average fitness value at each generation

    for gen in range(max_gen):
        # Evaluate fitness of each individual in the population
        fitness_values = [pop.Fitness for pop in population]
        avg_fitness = np.mean(fitness_values)
        avg_fitness_progress.append(avg_fitness)

        for i in range(NP):
            # Mutation (DE/current-to-best/1)
            r1, r2, r3 = np.random.choice([j for j in range(NP) if j != i], 3, replace=False)
            best_index = np.argmin([pop.Fitness for pop in population])
            mutant_genes = population[i].Genes + F * (population[best_index].Genes - population[i].Genes) + \
                           F * (population[r1].Genes - population[r2].Genes)

            # Crossover (intermediate)
            cross_points = np.random.rand(dim) < CR
            if not np.any(cross_points):
                cross_points[np.random.randint(dim)] = True
            trial_genes = np.where(cross_points, mutant_genes, population[i].Genes)
            trial_fitness = fitness_function(trial_genes)

            # Selection
            if trial_fitness < population[i].Fitness:
                population[i] = chromosome(trial_genes, trial_fitness)
                memory_CR.append(CR)
                memory_F.append(F)

        # Update CR and F
        if memory_CR:
            CR = (1 - 0.1) * CR + 0.1 * np.random.choice(memory_CR)
        if memory_F:
            F = (1 - 0.1) * F + 0.1 * np.random.choice(memory_F)
        
        # Record the best fitness value at this generation
        best_fitness = min(population, key=lambda x: x.Fitness).Fitness
        best_fitness_progress.append(best_fitness)

    # Plot the best fitness value over generations
    plt.plot(range(1, max_gen + 1), best_fitness_progress, label='Best Fitness')
    plt.plot(range(1, max_gen + 1), avg_fitness_progress, label='Average Fitness')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.title('Progression of JADE Algorithm')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Return the final best solution
    return min(population, key=lambda x: x.Fitness)


In [49]:
jade(fitness_function,4019200,300,0.9,0.8,4000)

/tmp/ipykernel_128/2387734017.py:39: RuntimeWarning: invalid value encountered in log
  loss = -np.mean(actual * np.log(predicted + epsilon))


KeyboardInterrupt: 

In [50]:
import tensorflow as tf

# Check available GPU devices
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

# Explicitly place operations on GPU
with tf.device('/GPU:0'):
    # Your TensorFlow code here
    # This part of the code will run on the GPU


SyntaxError: incomplete input (21984592.py, line 9)